In [298]:
import psycopg2 
import pandas as pd
import numpy as np
from pprint import pprint
import pandas as pd
from prophet import Prophet
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import joblib
from prophet_forecaster import ProphetForecaster

In [14]:
conn = psycopg2.connect(
    host="fit3164db.cxkhqsoitzhb.ap-southeast-2.rds.amazonaws.com", 
    port=5432,
    user="postgres",
    password="fit3164d13edas",
    database = "testdb"
)
conn.autocommit = True
cursor = conn.cursor()

In [33]:
#getting column names
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'actuals'")
cols = cursor.fetchall()
cols = [item[0] for item in cols]
print(cols)

['id', 'datetime', 'load', 'pressure', 'cloud_cover', 'humidity', 'temperature', 'wind_speed', 'wind_direction', 'date', 'month', 'hour', 'type_of_day', 'covid', 'holiday']


In [18]:
cursor.execute("select * from actuals")

In [19]:
retrieved_actual_data = cursor.fetchall()

In [49]:
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'forecasts'")
cols = cursor.fetchall()
cols = [item[0] for item in cols]
print(cols)

['id', 'datetime', 'pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction', 'date', 'month', 'hour', 'type_of_day', 'covid', 'holiday']


In [48]:
cursor.execute("select * from forecasts")
retrieved_forecast_data = cursor.fetchall()

In [ ]:
cursor.close()
conn.close()

In [36]:
actuals = pd.DataFrame(retrieved_actual_data, columns= cols)

In [43]:
actuals.dtypes

id                         int64
datetime          datetime64[ns]
load                     float64
pressure                 float64
cloud_cover              float64
humidity                 float64
temperature              float64
wind_speed               float64
wind_direction           float64
date                      object
month                      int64
hour                       int64
type_of_day                int64
covid                      int64
holiday                    int64
dtype: object

In [59]:
# converting data types:
def convert_dtypes(df, actual=True):
    if actual:
        numeric_cols = ['month','hour', 'type_of_day','covid', 'holiday', 'load', 'pressure', 'cloud_cover', 'humidity', 'temperature', 'wind_speed', 'wind_direction']
    else:
        numeric_cols = ['month','hour', 'type_of_day','covid', 'holiday', 'pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction']
    date_cols = ['datetime', 'date']
    
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])
        
    for col in date_cols:
        df[col] = pd.to_datetime(df[col])
    
    return df
actuals = convert_dtypes(actuals)
actuals

,id,datetime,load,pressure,cloud_cover,humidity,temperature,wind_speed,wind_direction,date,month,hour,type_of_day,covid,holiday
0,1,2017-03-18 00:00:00,1.031472e+06,1012.48000,2.125,79.33333,16.76439,6.25375,285.250,2017-03-18,3,0,6,0,0
1,2,2017-03-18 01:00:00,1.007206e+06,1011.61875,0.000,82.16667,15.72939,9.20500,285.000,2017-03-18,3,1,6,0,0
2,3,2017-03-18 02:00:00,9.861084e+05,1010.72000,2.125,81.66667,15.81689,11.28375,316.000,2017-03-18,3,2,6,0,0
3,4,2017-03-18 03:00:00,9.707610e+05,1011.37625,17.125,82.25000,15.91939,7.16875,327.000,2017-03-18,3,3,6,0,0
4,5,2017-03-18 04:00:00,9.622584e+05,1011.50000,18.625,82.33333,15.76939,0.78000,282.250,2017-03-18,3,4,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33595,33596,2021-01-15 19:00:00,1.141079e+06,1013.38750,38.375,68.33333,11.73439,6.63375,306.875,2021-01-15,1,19,5,1,0
33596,33597,2021-01-15 20:00:00,1.108896e+06,1013.97500,28.125,73.91667,9.05689,10.08750,297.625,2021-01-15,1,20,5,1,0
33597,33598,2021-01-15 21:00:00,1.078978e+06,1014.18750,21.125,75.83333,8.41439,8.18000,334.000,2021-01-15,1,21,5,1,0
33598,33599,2021-01-15 22:00:00,1.044495e+06,1014.12500,17.375,74.25000,8.04189,4.12250,73.625,2021-01-15,1,22,5,1,0


In [51]:
forecasts = pd.DataFrame(retrieved_forecast_data, columns= cols)

In [60]:
forecasts = convert_dtypes(forecasts, False)
forecasts.dtypes

id                         int64
datetime          datetime64[ns]
pressure                 float64
cloud_cover              float64
temperature              float64
wind_speed               float64
wind_direction           float64
date              datetime64[ns]
month                      int64
hour                       int64
type_of_day                int64
covid                      int64
holiday                    int64
dtype: object

In [62]:
forecasts

,id,datetime,pressure,cloud_cover,temperature,wind_speed,wind_direction,date,month,hour,type_of_day,covid,holiday
0,1,2017-03-18 00:00:00,1011.0,3.0,14.00000,2.0,307.0,2017-03-18,3,0,6,0,0
1,2,2017-03-18 01:00:00,1011.0,4.0,14.00000,2.0,215.0,2017-03-18,3,1,6,0,0
2,3,2017-03-18 02:00:00,1011.0,4.0,14.00000,1.0,123.0,2017-03-18,3,2,6,0,0
3,4,2017-03-18 03:00:00,1011.0,4.0,14.00000,1.0,31.0,2017-03-18,3,3,6,0,0
4,5,2017-03-18 04:00:00,1011.0,4.0,14.00000,1.0,138.0,2017-03-18,3,4,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33595,33596,2021-01-15 19:00:00,1013.0,0.0,15.72189,14.0,290.0,2021-01-15,1,19,5,1,0
33596,33597,2021-01-15 20:00:00,1013.0,0.0,13.72189,13.0,262.0,2021-01-15,1,20,5,1,0
33597,33598,2021-01-15 21:00:00,1014.0,0.0,11.72189,12.0,233.0,2021-01-15,1,21,5,1,0
33598,33599,2021-01-15 22:00:00,1014.0,1.0,9.72189,11.0,160.0,2021-01-15,1,22,5,1,0


Now we have retrieved the data from the database and changed the data types, we can play with it now:

In [165]:
def calculate_mae(actual, predicted):
    return mean_absolute_error(actual, predicted)

def calculate_mape(actual, predicted):
    a = actual.to_numpy()
    p = predicted.to_numpy()

    return np.mean(np.abs((a - p) / a)) * 100

def calculate_rmse(actual, predicted):
    return math.sqrt(mean_squared_error(actual, predicted))

def get_error_measures(actual, predicted):
    result = "MAE: " + str(calculate_mae(actual, predicted)) + "\n"
    result += "MAPE: " + str(calculate_mape(actual, predicted)) + "\n"
    result += "RMSE: " + str(calculate_rmse(actual, predicted)) + "\n"
    return result

In [183]:
MAE_THRESHOLD = 0.1
MAPE_THRESHOLD = 10
RMSE_THRESHOLD = 0.1

In [168]:
errors_df = pd.DataFrame(columns = ["start_date", "end_date", "mae", "mape", "rmse"])

In [92]:
data = pd.merge(actuals[['datetime', 'load']], forecasts, on='datetime')
data = data.rename(columns = {'datetime': 'ds','load': 'y'})

In [93]:
data.columns

Index(['ds', 'y', 'id', 'pressure', 'cloud_cover', 'temperature', 'wind_speed',
       'wind_direction', 'date', 'month', 'hour', 'type_of_day', 'covid',
       'holiday'],
      dtype='object')

In [94]:
train = data[:int((len(data)+1)*.8)]
test = data[int((len(data)+1)*.8):]

In [76]:
regressors = ['pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction', 'date', 'month', 'hour', 'type_of_day', 'covid', 'holiday']

In [73]:
model = Prophet(interval_width=0.95, changepoint_range=0.9, changepoint_prior_scale = 0.01,
 seasonality_prior_scale = 0.01)

In [96]:
model.fit(train)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


In [97]:
test_dfs = [test[i:i + 48] for i in range(0, len(test), 48)]

In [176]:
preds = model.predict(test_dfs[0])
yhat = preds['yhat']

In [115]:
actuals = data[data['ds'].isin(preds['ds'])]['y']

In [181]:
start_date = preds['ds'][0]
end_date = preds['ds'][len(preds)-1]
mae = calculate_mae(actuals, yhat)
mape = calculate_mape(actuals, yhat)
rmse = calculate_rmse(actuals, yhat)
new_row = {'start_date': start_date, 'end_date': end_date, 'mae': mae, 'mape': mape, 'rmse': rmse}
errors_df = pd.concat([errors_df, pd.DataFrame([new_row])], ignore_index=True)
errors_df.head()

,start_date,end_date,mae,mape,rmse
0,2020-04-11,2020-04-12 23:00:00,109232.081933,10.74574,130184.593626


In [184]:
preds = model.predict(test_dfs[1])
yhat = preds['yhat']
actuals = data[data['ds'].isin(preds['ds'])]['y']
start_date = preds['ds'][0]
end_date = preds['ds'][len(preds)-1]
mae = calculate_mae(actuals, yhat)
mape = calculate_mape(actuals, yhat)
rmse = calculate_rmse(actuals, yhat)
new_row = {'start_date': start_date, 'end_date': end_date, 'mae': mae, 'mape': mape, 'rmse': rmse}
errors_df = pd.concat([errors_df, pd.DataFrame([new_row])], ignore_index=True)
errors_df.head()

,start_date,end_date,mae,mape,rmse
0,2020-04-11,2020-04-12 23:00:00,109232.081933,10.745740,130184.593626
1,2020-04-13,2020-04-14 23:00:00,91256.349583,8.827931,103382.254010


In [204]:
preds = model.predict(test_dfs[2])
yhat = preds['yhat']
actuals = data[data['ds'].isin(preds['ds'])]['y']
start_date = preds['ds'][0]
end_date = preds['ds'][len(preds)-1]
mae = calculate_mae(actuals, yhat)
mape = calculate_mape(actuals, yhat)
rmse = calculate_rmse(actuals, yhat)
new_row = {'start_date': start_date, 'end_date': end_date, 'mae': mae, 'mape': mape, 'rmse': rmse}
errors_df = pd.concat([errors_df, pd.DataFrame([new_row])], ignore_index=True)
errors_df.head()

,start_date,end_date,mae,mape,rmse
0,2020-04-11,2020-04-12 23:00:00,109232.081933,10.745740,130184.593626
1,2020-04-13,2020-04-14 23:00:00,91256.349583,8.827931,103382.254010
2,2020-04-15,2020-04-16 23:00:00,87353.909554,8.301816,98762.914582


In [205]:
preds = model.predict(test_dfs[3])
yhat = preds['yhat']
actuals = data[data['ds'].isin(preds['ds'])]['y']
start_date = preds['ds'][0]
end_date = preds['ds'][len(preds)-1]
mae = calculate_mae(actuals, yhat)
mape = calculate_mape(actuals, yhat)
rmse = calculate_rmse(actuals, yhat)
new_row = {'start_date': start_date, 'end_date': end_date, 'mae': mae, 'mape': mape, 'rmse': rmse}
errors_df = pd.concat([errors_df, pd.DataFrame([new_row])], ignore_index=True)
errors_df.head()

,start_date,end_date,mae,mape,rmse
0,2020-04-11,2020-04-12 23:00:00,109232.081933,10.745740,130184.593626
1,2020-04-13,2020-04-14 23:00:00,91256.349583,8.827931,103382.254010
2,2020-04-15,2020-04-16 23:00:00,87353.909554,8.301816,98762.914582
3,2020-04-17,2020-04-18 23:00:00,101252.055139,9.888181,115467.652203


In [293]:
class ProphetForecaster:
    
    THRESHOLD_MAE = -0.1
    THRESHOLD_MAPE = -10
    THRESHOLD_RMSE = -0.1
    
    def __init__(self):
        self.model = None
        self.data = None
        self.regressors = None
        
        self.errors = pd.DataFrame(columns = ["start_date", "end_date", "mae", "mape", "rmse"])
        
        
    def initialise(self, train, data, regressors):
        self.model = Prophet(interval_width=0.95, changepoint_range=0.9, changepoint_prior_scale = 0.01,
 seasonality_prior_scale = 0.01)
        self.data = data
        self.regressors = regressors
        
        for reg in self.regressors:
            self.model.add_regressor(reg)
            
        # in reality we want to get the data from the database and then store it
        self.data = self.data.rename(columns = {'datetime': 'ds',
                        'load': 'y'})
        
        self._fit()
        
        
    def _fit(self):
        self.model.fit(self.data)
        
    def predict(self, new_data):
        
#         self.data = pd.concat([self.data, new_data], ignore_index = True)
        
        predictions = self.model.predict(new_data)
        yhat = predictions['yhat']
        actuals = self.data[self.data['ds'].isin(predictions['ds'])]['y']
        start_date = predictions['ds'].iloc[0]
        end_date = predictions['ds'].iloc[-1]
        mae = self.calculate_mae(actuals, yhat)
        mape = self.calculate_mape(actuals, yhat)
        rmse = self.calculate_rmse(actuals, yhat)
        new_row = {'start_date': start_date, 'end_date': end_date, 'mae': mae, 'mape': mape, 'rmse': rmse}
        self.errors = pd.concat([self.errors, pd.DataFrame([new_row])], ignore_index=True)
                
        if self.check_for_errors_changes():
            self.retrain()
            
        return predictions
    
    def retrain(self):
        print("retraining")
        
        #get the data, could be all of it or a subset
        self.model = Prophet(interval_width=0.95, changepoint_range=0.9, changepoint_prior_scale = 0.01,
 seasonality_prior_scale = 0.01)
        for reg in self.regressors:
            self.model.add_regressor(reg)
        self._fit()
        self.errors.drop(self.errors.index, inplace=True)
        print("retrained")
          
        
    def check_for_errors_changes(self):
        
        initial = self.errors.iloc[0]
        last = self.errors.iloc[-1]
        
        mae_change = (initial.mae  - last.mae) / initial.mae
        mape_change = (initial.mape  - last.mape)
        rmse_change = (initial.rmse  - last.rmse) / initial.rmse

        print(mae_change)
        print(mape_change)
        print(rmse_change)
        
        if mape_change < 0 and rmse_change < 0 and mae_change < ProphetForecaster.THRESHOLD_MAE and \
        mape_change < ProphetForecaster.THRESHOLD_MAPE and rmse_change < ProphetForecaster.THRESHOLD_RMSE:
            return True
        else:
            return False
        
         
    def calculate_mae(self, actual, predicted):
        return mean_absolute_error(actual, predicted)

    def calculate_mape(self, actual, predicted):
        a = actual.to_numpy()
        p = predicted.to_numpy()

        return np.mean(np.abs((a - p) / a)) * 100

    def calculate_rmse(self, actual, predicted):
        return math.sqrt(mean_squared_error(actual, predicted))


    def get_error_measures(self, actual, predicted):
        result = "MAE: " + str(calculate_mae(actual, predicted)) + "\n"
        result += "MAPE: " + str(calculate_mape(actual, predicted)) + "\n"
        result += "RMSE: " + str(calculate_rmse(actual, predicted)) + "\n"
        return result        

In [294]:
model = ProphetForecaster()
model.initialise(train, data, regressors)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


In [297]:
joblib.dump(model, 'prophest_test.joblib')

['prophest_test.joblib']

### Testing joblib:

In [299]:
model = joblib.load('prophest_test.joblib')

In [302]:
model.predict(test_dfs[2])

0.15889911226093592
1.7956684617992495
0.16597766013250392


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,cloud_cover,...,wind_speed,wind_speed_lower,wind_speed_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-04-15 00:00:00,1.157761e+06,8.444336e+05,1.097987e+06,1.157761e+06,1.157761e+06,-181887.923057,-181887.923057,-181887.923057,3961.508385,...,332.785309,332.785309,332.785309,35061.666679,35061.666679,35061.666679,0.0,0.0,0.0,9.758729e+05
1,2020-04-15 01:00:00,1.157771e+06,8.384249e+05,1.074583e+06,1.157771e+06,1.157771e+06,-203662.037681,-203662.037681,-203662.037681,3961.508385,...,332.785309,332.785309,332.785309,35046.159894,35046.159894,35046.159894,0.0,0.0,0.0,9.541089e+05
2,2020-04-15 02:00:00,1.157781e+06,8.134729e+05,1.061195e+06,1.157781e+06,1.157781e+06,-221539.572977,-221539.572977,-221539.572977,3961.508385,...,514.241235,514.241235,514.241235,35030.469751,35030.469751,35030.469751,0.0,0.0,0.0,9.362415e+05
3,2020-04-15 03:00:00,1.157791e+06,8.053901e+05,1.045511e+06,1.157791e+06,1.157791e+06,-231400.270329,-231400.270329,-231400.270329,3961.508385,...,695.697161,695.697161,695.697161,35014.597819,35014.597819,35014.597819,0.0,0.0,0.0,9.263909e+05
4,2020-04-15 04:00:00,1.157801e+06,7.954347e+05,1.036454e+06,1.157801e+06,1.157801e+06,-239781.312230,-239781.312230,-239781.312230,3807.784395,...,1058.609014,1058.609014,1058.609014,34998.545681,34998.545681,34998.545681,0.0,0.0,0.0,9.180200e+05
5,2020-04-15 05:00:00,1.157811e+06,7.987140e+05,1.043035e+06,1.157811e+06,1.157811e+06,-234484.678599,-234484.678599,-234484.678599,3807.784395,...,1421.520867,1421.520867,1421.520867,34982.314934,34982.314934,34982.314934,0.0,0.0,0.0,9.233267e+05
6,2020-04-15 06:00:00,1.157822e+06,8.255167e+05,1.066260e+06,1.157822e+06,1.157822e+06,-212474.119950,-212474.119950,-212474.119950,3807.784395,...,1784.432720,1784.432720,1784.432720,34965.907186,34965.907186,34965.907186,0.0,0.0,0.0,9.453474e+05
7,2020-04-15 07:00:00,1.157832e+06,8.638368e+05,1.118299e+06,1.157832e+06,1.157832e+06,-162043.851296,-162043.851296,-162043.851296,3807.784395,...,1784.432720,1784.432720,1784.432720,34949.324058,34949.324058,34949.324058,0.0,0.0,0.0,9.957878e+05
8,2020-04-15 08:00:00,1.157842e+06,9.438095e+05,1.201242e+06,1.157842e+06,1.157842e+06,-89830.202954,-89830.202954,-89830.202954,3807.784395,...,1784.432720,1784.432720,1784.432720,34932.567184,34932.567184,34932.567184,0.0,0.0,0.0,1.068012e+06
9,2020-04-15 09:00:00,1.157852e+06,1.016760e+06,1.266729e+06,1.157852e+06,1.157852e+06,-14459.762475,-14459.762475,-14459.762475,3654.060406,...,1784.432720,1784.432720,1784.432720,34915.638212,34915.638212,34915.638212,0.0,0.0,0.0,1.143392e+06
